In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Customer List.csv
/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Geolocations.csv
/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Sellers List.csv
/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Order Payments.csv
/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Products.csv
/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Order Items.csv
/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Orders.csv
/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom_Inc_Order_Reviews_No_Emojis.csv


In [8]:
# Caminhos dos arquivos
file_paths = {
    "customers": "/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Customer List.csv",
    "geolocations": "/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Geolocations.csv",
    "order_items": "/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Order Items.csv",
    "order_payments": "/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Order Payments.csv",
    "orders": "/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Orders.csv",
    "products": "/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Products.csv",
    "sellers": "/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom Inc Sellers List.csv",
    "order_reviews": "/kaggle/input/fecom-inc-e-com-marketplace-orders-data-crm/Fecom_Inc_Order_Reviews_No_Emojis.csv",
}

# Carregar os arquivos corretamente com ';' como delimitador
dataframes = {}
for name, path in file_paths.items():
    try:
        df = pd.read_csv(path, sep=';', encoding='utf-8', low_memory=False)
        dataframes[name] = df
    except Exception as e:
        dataframes[name] = None  # Define como None se houver erro
        print(f"Erro ao carregar {name}: {e}")

# Converter colunas de data para formato datetime
date_columns = {
    "customers": ["Subscribe_Date", "First_Order_Date"],
    "order_items": ["Shipping_Limit_Date"],
    "orders": ["Order_Purchase_Timestamp", "Order_Approved_At", 
               "Order_Delivered_Carrier_Date", "Order_Delivered_Customer_Date", 
               "Order_Estimated_Delivery_Date"],
    "order_reviews": ["Review_Creation_Date", "Review_Answer_Timestamp"]
}

for dataset, columns in date_columns.items():
    if dataset in dataframes and dataframes[dataset] is not None:
        for col in columns:
            if col in dataframes[dataset].columns:
                dataframes[dataset][col] = pd.to_datetime(dataframes[dataset][col], errors='coerce')

# Preenchendo valores ausentes apropriados
if dataframes["geolocations"] is not None:
    dataframes["geolocations"].fillna({"Geo_Lat": 0, "Geo_Lon": 0, "Geolocation_City": "Unknown", "Geo_Country": "Unknown"}, inplace=True)
if dataframes["products"] is not None:
    dataframes["products"].fillna({"Product_Category_Name": "Unknown"}, inplace=True)
if dataframes["orders"] is not None:
    dataframes["orders"].fillna({"Order_Approved_At": dataframes["orders"]["Order_Purchase_Timestamp"],
                                 "Order_Delivered_Carrier_Date": dataframes["orders"]["Order_Estimated_Delivery_Date"],
                                 "Order_Delivered_Customer_Date": dataframes["orders"]["Order_Estimated_Delivery_Date"]}, inplace=True)
if dataframes["order_reviews"] is not None:
    dataframes["order_reviews"].fillna({"Review_Comment_Title_En": "No Title", "Review_Comment_Message_En": "No Comment"}, inplace=True)

# Criar novas colunas derivadas
if dataframes["orders"] is not None:
    # 1. Tempo de entrega do pedido (dias entre a compra e a entrega ao cliente)
    dataframes["orders"]["Delivery_Duration_Days"] = (dataframes["orders"]["Order_Delivered_Customer_Date"] - 
                                                      dataframes["orders"]["Order_Purchase_Timestamp"]).dt.days

if dataframes["order_items"] is not None:
    # 2. Criar uma coluna de categoria de preço para os produtos
    bins = [0, 50, 200, 500, 1000, 5000]
    labels = ["Muito Barato", "Barato", "Médio", "Caro", "Muito Caro"]
    dataframes["order_items"]["Price_Category"] = pd.cut(dataframes["order_items"]["Price"], bins=bins, labels=labels)

# Exibir informações gerais dos datasets carregados
for name, df in dataframes.items():
    if df is not None:
        print(f"\n{name} dataset:")
        print(df.info())

# Limpeza e preparação dos dados
if dataframes["geolocations"] is not None:
    dataframes["geolocations"].fillna({"Geo_Lat": 0, "Geo_Lon": 0, "Geolocation_City": "Unknown", "Geo_Country": "Unknown"}, inplace=True)
if dataframes["products"] is not None:
    dataframes["products"].fillna({"Product_Category_Name": "Unknown"}, inplace=True)
    cols_to_fill = ["Product_Weight_Gr", "Product_Length_Cm", "Product_Height_Cm", "Product_Width_Cm"]
    for col in cols_to_fill:
        dataframes["products"][col].fillna(dataframes["products"][col].median(), inplace=True)
if dataframes["orders"] is not None:
    dataframes["orders"].fillna({"Order_Approved_At": dataframes["orders"]["Order_Purchase_Timestamp"],
                                 "Order_Delivered_Carrier_Date": dataframes["orders"]["Order_Estimated_Delivery_Date"],
                                 "Order_Delivered_Customer_Date": dataframes["orders"]["Order_Estimated_Delivery_Date"]}, inplace=True)
if dataframes["order_reviews"] is not None:
    dataframes["order_reviews"].fillna({"Review_Comment_Title_En": "No Title", "Review_Comment_Message_En": "No Comment"}, inplace=True)

# Conversão de colunas de data
date_columns = {
    "customers": ["Subscribe_Date", "First_Order_Date"],
    "order_items": ["Shipping_Limit_Date"],
    "orders": ["Order_Purchase_Timestamp", "Order_Approved_At", 
               "Order_Delivered_Carrier_Date", "Order_Delivered_Customer_Date", 
               "Order_Estimated_Delivery_Date"],
    "order_reviews": ["Review_Creation_Date", "Review_Answer_Timestamp"]
}

for dataset, columns in date_columns.items():
    if dataset in dataframes and dataframes[dataset] is not None:
        for col in columns:
            if col in dataframes[dataset].columns:
                dataframes[dataset][col] = pd.to_datetime(dataframes[dataset][col], errors='coerce')

# Criar novas colunas derivadas
if dataframes["orders"] is not None:
    dataframes["orders"]["Delivery_Duration_Days"] = (dataframes["orders"]["Order_Delivered_Customer_Date"] - 
                                                      dataframes["orders"]["Order_Purchase_Timestamp"]).dt.days

if dataframes["order_items"] is not None:
    bins = [0, 50, 200, 500, 1000, 5000]
    labels = ["Muito Barato", "Barato", "Médio", "Caro", "Muito Caro"]
    dataframes["order_items"]["Price_Category"] = pd.cut(dataframes["order_items"]["Price"], bins=bins, labels=labels)

# Exibir informações gerais dos datasets processados
for name, df in dataframes.items():
    if df is not None:
        print(f"\n{name} dataset:")
        print(df.info())


customers dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102727 entries, 0 to 102726
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Customer_Trx_ID        99441 non-null   object        
 1   Subscriber_ID          102727 non-null  object        
 2   Subscribe_Date         102727 non-null  datetime64[ns]
 3   First_Order_Date       99441 non-null   datetime64[ns]
 4   Customer_Postal_Code   102727 non-null  object        
 5   Customer_City          102727 non-null  object        
 6   Customer_Country       102727 non-null  object        
 7   Customer_Country_Code  102727 non-null  object        
 8   Age                    102727 non-null  int64         
 9   Gender                 102727 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 7.8+ MB
None

geolocations dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10

<ipython-input-8-28e766d5521c>:76: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataframes["products"][col].fillna(dataframes["products"][col].median(), inplace=True)



customers dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102727 entries, 0 to 102726
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Customer_Trx_ID        99441 non-null   object        
 1   Subscriber_ID          102727 non-null  object        
 2   Subscribe_Date         102727 non-null  datetime64[ns]
 3   First_Order_Date       99441 non-null   datetime64[ns]
 4   Customer_Postal_Code   102727 non-null  object        
 5   Customer_City          102727 non-null  object        
 6   Customer_Country       102727 non-null  object        
 7   Customer_Country_Code  102727 non-null  object        
 8   Age                    102727 non-null  int64         
 9   Gender                 102727 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 7.8+ MB
None

geolocations dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10

In [ ]:
import matplotlib.pyplot as plt

# Conversão de colunas de latitude e longitude para float
if dataframes["geolocations"] is not None:
    dataframes["geolocations"]["Geo_Lat"] = pd.to_numeric(dataframes["geolocations"]["Geo_Lat"], errors='coerce')
    dataframes["geolocations"]["Geo_Lon"] = pd.to_numeric(dataframes["geolocations"]["Geo_Lon"], errors='coerce')

# Criar visualizações
figures = []

# 1. Gráfico de barras - Quantidade de pedidos por status
if dataframes["orders"] is not None:
    order_status_counts = dataframes["orders"]["Order_Status"].value_counts()
    fig, ax = plt.subplots(figsize=(8, 5))
    order_status_counts.plot(kind="bar", ax=ax)
    ax.set_title("Quantidade de Pedidos por Status")
    ax.set_xlabel("Status do Pedido")
    ax.set_ylabel("Quantidade")
    figures.append(fig)

# 2. Gráfico de linha - Evolução do número de pedidos ao longo do tempo
if dataframes["orders"] is not None:
    orders_by_date = dataframes["orders"].groupby(dataframes["orders"]["Order_Purchase_Timestamp"].dt.date).size()
    fig, ax = plt.subplots(figsize=(10, 5))
    orders_by_date.plot(kind="line", ax=ax)
    ax.set_title("Evolução dos Pedidos ao Longo do Tempo")
    ax.set_xlabel("Data")
    ax.set_ylabel("Número de Pedidos")
    figures.append(fig)

# 3. Gráfico de dispersão - Relação entre preço e valor do frete
if dataframes["order_items"] is not None:
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.scatter(dataframes["order_items"]["Price"], dataframes["order_items"]["Freight_Value"], alpha=0.5)
    ax.set_title("Relação entre Preço do Produto e Valor do Frete")
    ax.set_xlabel("Preço do Produto")
    ax.set_ylabel("Valor do Frete")
    figures.append(fig)

# 4. Mapa - Distribuição geográfica dos vendedores
if dataframes["geolocations"] is not None:
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.scatter(dataframes["geolocations"]["Geo_Lon"], dataframes["geolocations"]["Geo_Lat"], alpha=0.3, marker=".")
    ax.set_title("Distribuição Geográfica dos Vendedores")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    figures.append(fig)

# 5. Tabela dinâmica - Média de valor gasto por número de parcelas
if dataframes["order_payments"] is not None:
    pivot_table = dataframes["order_payments"].groupby("Payment_Installments")["Payment_Value"].mean().reset_index()
    print("\nMédia de Pagamento por Número de Parcelas:")
    print(pivot_table)

# Exibir os gráficos gerados
for fig in figures:
    plt.show(fig)